In [27]:
from Cleaner import *
from config import db_password

from sqlalchemy import create_engine
from pathlib import Path
import numpy as np
import pandas as pd
import psycopg2

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [28]:
awards = pd.read_csv('Resources/AwardsPlayers.csv')
all_awards = reverseLookup(awards)

In [29]:
data_NL, data_AL = Merger(all_awards)
data_NL

............Scraping Data.............

1982 Time: 1.450941801071167 Seconds
1983 Time: 1.2226152420043945 Seconds
1984 Time: 1.214066982269287 Seconds
1985 Time: 1.2571690082550049 Seconds
1986 Time: 1.168247938156128 Seconds

.....Approximately 49-62 Seconds.....

Total Time: 60.61354303359985 Seconds


,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,CSW%,xBA,xSLG,xwOBA,key_fangraphs,playerID,awardID,yearID,lgID,MVP
6,1002015,1982,Gary Carter,MON,28,154,557,653,163,101,...,NaN,NaN,NaN,NaN,1002015.0,cartega01,Gold Glove,1982.0,NL,0
7,1002015,1982,Gary Carter,MON,28,154,557,653,163,101,...,NaN,NaN,NaN,NaN,1002015.0,cartega01,Silver Slugger,1982.0,NL,0
8,1002015,1982,Gary Carter,MON,28,154,557,653,163,101,...,NaN,NaN,NaN,NaN,1002015.0,cartega01,TSN All-Star,1982.0,NL,0
9,1003091,1982,Andre Dawson,MON,27,148,607,659,183,116,...,NaN,NaN,NaN,NaN,1003091.0,dawsoan01,Gold Glove,1982.0,NL,0
12,1011586,1982,Mike Schmidt,PHI,32,148,514,631,144,80,...,NaN,NaN,NaN,NaN,1011586.0,schmimi01,Gold Glove,1982.0,NL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,3516,2021,Eric Hosmer,SDP,31,151,509,565,137,97,...,0.263,0.268,0.406,0.326,NaN,NaN,No Award,NaN,NL,0
141,7859,2021,Charlie Blackmon,COL,34,150,514,582,139,97,...,0.250,0.291,0.450,0.359,NaN,NaN,No Award,NaN,NL,0
142,19892,2021,Pavin Smith,ARI,25,145,498,545,133,91,...,0.282,0.258,0.402,0.317,NaN,NaN,No Award,NaN,NL,0
144,17696,2021,Kevin Newman,PIT,27,148,517,554,117,87,...,0.255,0.255,0.330,0.275,NaN,NaN,No Award,NaN,NL,0


## START NL STATS

In [30]:
# Determined that some stats were fielding, pitching, or otherwise irrelevant to our task at hand.

data_NL = data_NL[[ 'G',
 'AB',
 'PA',
 'H',
 '1B',
 '2B',
 '3B',
 'HR',
 'R',
 'RBI',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'SF',
 'SH',
 'SB',
 'AVG',
 'OBP',
 'SLG', 
 'OPS',
 'ISO',
 'BABIP',
 'wOBA',
 'wRAA',
 'wRC',
 'WAR',
 'Spd',
 'wRC+',
 'WPA',
 '-WPA',
 '+WPA',
 'RE24',
 'REW',
 'pLI',
 'WPA/LI',
 'Clutch',
 'BsR',
 'wSB',
 'Off',
 'AVG+',
 'OBP+',
 'SLG+',
 'ISO+',
 'BABIP+',
 'MVP']]

data_NL

,G,AB,PA,H,1B,2B,3B,HR,R,RBI,...,Clutch,BsR,wSB,Off,AVG+,OBP+,SLG+,ISO+,BABIP+,MVP
6,154,557,653,163,101,32,1,29,91,97,...,-1.05,-1.6,-1.6,34.9,110,116,132,181,98,0
7,154,557,653,163,101,32,1,29,91,97,...,-1.05,-1.6,-1.6,34.9,110,116,132,181,98,0
8,154,557,653,163,101,32,1,29,91,97,...,-1.05,-1.6,-1.6,34.9,110,116,132,181,98,0
9,148,607,659,183,116,37,7,23,107,83,...,-0.85,3.9,3.9,30.8,114,105,129,164,112,0
12,148,514,631,144,80,26,3,35,108,87,...,-0.47,0.1,0.1,42.9,105,122,142,222,106,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,151,509,565,137,97,28,0,12,53,65,...,0.32,-5.5,-1.2,-3.9,108,103,94,73,106,0
141,150,514,582,139,97,25,4,13,76,78,...,0.52,1.4,0.0,-2.7,109,107,98,82,104,0
142,145,498,545,133,91,27,4,11,68,49,...,-0.13,-2.4,-0.3,-4.9,107,101,96,80,108,0
144,148,517,554,117,87,22,3,5,50,39,...,0.79,5.1,0.4,-28.2,91,81,74,49,80,0


In [31]:
# Remove duplicate rows that were present due to players winning multiple awards in one year (see original, raw dataframe)

data_NL = data_NL.drop_duplicates()
data_NL


,G,AB,PA,H,1B,2B,3B,HR,R,RBI,...,Clutch,BsR,wSB,Off,AVG+,OBP+,SLG+,ISO+,BABIP+,MVP
6,154,557,653,163,101,32,1,29,91,97,...,-1.05,-1.6,-1.6,34.9,110,116,132,181,98,0
9,148,607,659,183,116,37,7,23,107,83,...,-0.85,3.9,3.9,30.8,114,105,129,164,112,0
12,148,514,631,144,80,26,3,35,108,87,...,-0.47,0.1,0.1,42.9,105,122,142,222,106,0
17,150,575,652,175,111,27,5,32,87,100,...,-0.98,2.4,2.4,43.2,115,115,139,192,108,0
19,162,598,698,168,107,23,2,36,113,109,...,0.50,0.3,0.3,34.9,106,115,131,188,106,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,151,509,565,137,97,28,0,12,53,65,...,0.32,-5.5,-1.2,-3.9,108,103,94,73,106,0
141,150,514,582,139,97,25,4,13,76,78,...,0.52,1.4,0.0,-2.7,109,107,98,82,104,0
142,145,498,545,133,91,27,4,11,68,49,...,-0.13,-2.4,-0.3,-4.9,107,101,96,80,108,0
144,148,517,554,117,87,22,3,5,50,39,...,0.79,5.1,0.4,-28.2,91,81,74,49,80,0


In [32]:

# Count instances of MVP winners to make sure our data makes sense 

data_NL['MVP'].value_counts()[1]

38

## START AL STATS

In [33]:
# Repeat the above cleaning on the data_AL data set
# Merge both data_NL and data_AL afterwards 

data_AL

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,CSW%,xBA,xSLG,xwOBA,key_fangraphs,playerID,awardID,yearID,lgID,MVP
0,1014396,1982,Robin Yount,MIL,26,156,635,704,210,123,...,NaN,NaN,NaN,NaN,1014396.0,yountro01,Gold Glove,1982.0,AL,1
1,1014396,1982,Robin Yount,MIL,26,156,635,704,210,123,...,NaN,NaN,NaN,NaN,1014396.0,yountro01,Most Valuable Player,1982.0,AL,1
2,1014396,1982,Robin Yount,MIL,26,156,635,704,210,123,...,NaN,NaN,NaN,NaN,1014396.0,yountro01,Silver Slugger,1982.0,AL,1
3,1014396,1982,Robin Yount,MIL,26,156,635,704,210,123,...,NaN,NaN,NaN,NaN,1014396.0,yountro01,TSN All-Star,1982.0,AL,1
4,1014396,1982,Robin Yount,MIL,26,156,635,704,210,123,...,NaN,NaN,NaN,NaN,1014396.0,yountro01,TSN Major League Player of the Year,1982.0,AL,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,15149,2021,Trey Mancini,BAL,29,147,556,616,142,87,...,0.287,0.266,0.452,0.341,NaN,NaN,No Award,NaN,AL,0
143,10243,2021,Randal Grichuk,TOR,29,149,511,545,123,75,...,0.249,0.234,0.402,0.297,NaN,NaN,No Award,NaN,AL,0
147,2396,2021,Carlos Santana,KCR,35,158,565,659,121,87,...,0.242,0.244,0.421,0.335,NaN,NaN,No Award,NaN,AL,0
148,1744,2021,Miguel Cabrera,DET,38,130,472,526,121,90,...,0.274,0.234,0.420,0.315,NaN,NaN,No Award,NaN,AL,0


In [34]:
# Dropped all columns with NaN values 

data_AL = data_AL.dropna(axis=1)
data_AL

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,Events,awardID,lgID,MVP
0,1014396,1982,Robin Yount,MIL,26,156,635,704,210,123,...,91,71,115,144,179,116,0,Gold Glove,AL,1
1,1014396,1982,Robin Yount,MIL,26,156,635,704,210,123,...,91,71,115,144,179,116,0,Most Valuable Player,AL,1
2,1014396,1982,Robin Yount,MIL,26,156,635,704,210,123,...,91,71,115,144,179,116,0,Silver Slugger,AL,1
3,1014396,1982,Robin Yount,MIL,26,156,635,704,210,123,...,91,71,115,144,179,116,0,TSN All-Star,AL,1
4,1014396,1982,Robin Yount,MIL,26,156,635,704,210,123,...,91,71,115,144,179,116,0,TSN Major League Player of the Year,AL,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,15149,2021,Trey Mancini,BAL,29,147,556,616,142,87,...,98,101,103,104,104,106,414,No Award,AL,0
143,10243,2021,Randal Grichuk,TOR,29,149,511,545,123,75,...,58,91,89,102,107,91,401,No Award,AL,0
147,2396,2021,Carlos Santana,KCR,35,158,565,659,121,87,...,154,68,101,82,75,78,468,No Award,AL,0
148,1744,2021,Miguel Cabrera,DET,38,130,472,526,121,90,...,90,98,100,93,76,104,363,No Award,AL,0


In [35]:
# Determined that some stats were fielding, pitching, or otherwise irrelevant to our task at hand.

data_AL = data_AL[[ 'G',
 'AB',
 'PA',
 'H',
 '1B',
 '2B',
 '3B',
 'HR',
 'R',
 'RBI',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'SF',
 'SH',
 'SB',
 'AVG',
 'OBP',
 'SLG', 
 'OPS',
 'ISO',
 'BABIP',
 'wOBA',
 'wRAA',
 'wRC',
 'WAR',
 'Spd',
 'wRC+',
 'WPA',
 '-WPA',
 '+WPA',
 'RE24',
 'REW',
 'pLI',
 'WPA/LI',
 'Clutch',
 'BsR',
 'wSB',
 'Off',
 'AVG+',
 'OBP+',
 'SLG+',
 'ISO+',
 'BABIP+',
 'MVP']]

data_AL

,G,AB,PA,H,1B,2B,3B,HR,R,RBI,...,Clutch,BsR,wSB,Off,AVG+,OBP+,SLG+,ISO+,BABIP+,MVP
0,156,635,704,210,123,46,12,29,129,114,...,-0.63,1.6,1.6,54.7,125,115,144,179,116,1
1,156,635,704,210,123,46,12,29,129,114,...,-0.63,1.6,1.6,54.7,125,115,144,179,116,1
2,156,635,704,210,123,46,12,29,129,114,...,-0.63,1.6,1.6,54.7,125,115,144,179,116,1
3,156,635,704,210,123,46,12,29,129,114,...,-0.63,1.6,1.6,54.7,125,115,144,179,116,1
4,156,635,704,210,123,46,12,29,129,114,...,-0.63,1.6,1.6,54.7,125,115,144,179,116,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,147,556,616,142,87,33,1,21,77,71,...,-0.35,-5.3,-0.5,-1.8,104,103,104,104,106,0
143,149,511,545,123,75,25,1,22,59,81,...,-1.14,-4.9,-1.6,-14.7,98,89,102,107,91,0
147,158,565,659,121,87,15,0,19,66,69,...,0.25,-0.8,-0.2,-15.3,87,101,82,75,78,0
148,130,472,526,121,90,16,0,15,48,75,...,0.53,-6.3,-0.5,-10.7,104,100,93,76,104,0


In [36]:
# Remove duplicate rows that were present due to players winning multiple awards in one year (see original, raw dataframe)

data_AL = data_AL.drop_duplicates()
data_AL

,G,AB,PA,H,1B,2B,3B,HR,R,RBI,...,Clutch,BsR,wSB,Off,AVG+,OBP+,SLG+,ISO+,BABIP+,MVP
0,156,635,704,210,123,46,12,29,129,114,...,-0.63,1.6,1.6,54.7,125,115,144,179,116,1
10,153,575,655,173,96,42,5,30,94,97,...,-1.68,-0.6,-0.6,35.7,114,112,136,179,107,0
15,162,609,727,178,102,37,7,32,122,98,...,-0.87,-0.2,-0.2,44.2,111,123,133,175,114,0
24,162,602,708,183,125,29,4,25,100,78,...,0.00,2.2,2.2,37.5,115,121,122,135,106,0
25,151,550,627,174,111,30,1,32,87,110,...,1.72,0.6,0.6,40.0,120,119,136,168,114,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,147,556,616,142,87,33,1,21,77,71,...,-0.35,-5.3,-0.5,-1.8,104,103,104,104,106,0
143,149,511,545,123,75,25,1,22,59,81,...,-1.14,-4.9,-1.6,-14.7,98,89,102,107,91,0
147,158,565,659,121,87,15,0,19,66,69,...,0.25,-0.8,-0.2,-15.3,87,101,82,75,78,0
148,130,472,526,121,90,16,0,15,48,75,...,0.53,-6.3,-0.5,-10.7,104,100,93,76,104,0


In [37]:
# Count instances of MVP winners to make sure our data makes sense 

data_AL['MVP'].value_counts()[1]

35

In [38]:
# Going back and adding Lindsay's new metric to each data_NL and data_AL
data_NL['XBH+'] = (((data_NL['R'] + data_NL['2B'] + data_NL['3B'] + data_NL['HR']) - (data_NL['SO']))/data_NL['PA'])
data_NL.head()

,G,AB,PA,H,1B,2B,3B,HR,R,RBI,...,BsR,wSB,Off,AVG+,OBP+,SLG+,ISO+,BABIP+,MVP,XBH+
6,154,557,653,163,101,32,1,29,91,97,...,-1.6,-1.6,34.9,110,116,132,181,98,0,0.136294
9,148,607,659,183,116,37,7,23,107,83,...,3.9,3.9,30.8,114,105,129,164,112,0,0.119879
12,148,514,631,144,80,26,3,35,108,87,...,0.1,0.1,42.9,105,122,142,222,106,0,0.064976
17,150,575,652,175,111,27,5,32,87,100,...,2.4,2.4,43.2,115,115,139,192,108,0,0.095092
19,162,598,698,168,107,23,2,36,113,109,...,0.3,0.3,34.9,106,115,131,188,106,1,0.057307


In [39]:
data_AL['XBH+'] = (((data_AL['R'] + data_AL['2B'] + data_AL['3B'] + data_AL['HR']) - (data_AL['SO']))/data_AL['PA'])
data_AL.head()

,G,AB,PA,H,1B,2B,3B,HR,R,RBI,...,BsR,wSB,Off,AVG+,OBP+,SLG+,ISO+,BABIP+,MVP,XBH+
0,156,635,704,210,123,46,12,29,129,114,...,1.6,1.6,54.7,125,115,144,179,116,1,0.217330
10,153,575,655,173,96,42,5,30,94,97,...,-0.6,-0.6,35.7,114,112,136,179,107,0,0.138931
15,162,609,727,178,102,37,7,32,122,98,...,-0.2,-0.2,44.2,111,123,133,175,114,0,0.100413
24,162,602,708,183,125,29,4,25,100,78,...,2.2,2.2,37.5,115,121,122,135,106,0,0.149718
25,151,550,627,174,111,30,1,32,87,110,...,0.6,0.6,40.0,120,119,136,168,114,0,0.108453


In [40]:
# Merge the final data_NL and data_AL datasets 
cleaned_merged_df = pd.concat([data_NL, data_AL], axis=0)
cleaned_merged_df

,G,AB,PA,H,1B,2B,3B,HR,R,RBI,...,BsR,wSB,Off,AVG+,OBP+,SLG+,ISO+,BABIP+,MVP,XBH+
6,154,557,653,163,101,32,1,29,91,97,...,-1.6,-1.6,34.9,110,116,132,181,98,0,0.136294
9,148,607,659,183,116,37,7,23,107,83,...,3.9,3.9,30.8,114,105,129,164,112,0,0.119879
12,148,514,631,144,80,26,3,35,108,87,...,0.1,0.1,42.9,105,122,142,222,106,0,0.064976
17,150,575,652,175,111,27,5,32,87,100,...,2.4,2.4,43.2,115,115,139,192,108,0,0.095092
19,162,598,698,168,107,23,2,36,113,109,...,0.3,0.3,34.9,106,115,131,188,106,1,0.057307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,147,556,616,142,87,33,1,21,77,71,...,-5.3,-0.5,-1.8,104,103,104,104,106,0,-0.017857
143,149,511,545,123,75,25,1,22,59,81,...,-4.9,-1.6,-14.7,98,89,102,107,91,0,-0.012844
147,158,565,659,121,87,15,0,19,66,69,...,-0.8,-0.2,-15.3,87,101,82,75,78,0,-0.003035
148,130,472,526,121,90,16,0,15,48,75,...,-6.3,-0.5,-10.7,104,100,93,76,104,0,-0.074144


In [41]:
# Count instances of MVP winners to make sure our data makes sense 

cleaned_merged_df['MVP'].value_counts()[1]

73

## Start ML for dataset

In [42]:
y = cleaned_merged_df['MVP'].values
X = cleaned_merged_df.drop(['MVP'],1).values

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [44]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [45]:
classifier = LogisticRegression(solver='lbfgs')
classifier

LogisticRegression()

In [48]:
# Train the data
classifier.fit(X_train, y_train)

LogisticRegression()

In [49]:
y_pred = classifier.predict(X_test)

In [50]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9862012987012987


In [51]:
# Import 2022 stats

from pybaseball import batting_stats

current_year = batting_stats(2022)

current_year

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
1,15640,2022,Aaron Judge,NYY,30,120,448,529,133,65,...,0.256,118.4,193,0.603,320,0.177,0.292,0.296,0.693,0.451
0,9218,2022,Paul Goldschmidt,STL,34,117,442,511,148,83,...,0.123,112.3,159,0.466,341,0.194,0.289,0.264,0.501,0.372
3,9777,2022,Nolan Arenado,STL,31,115,438,489,132,72,...,0.087,111.4,151,0.396,381,0.164,0.251,0.267,0.451,0.341
4,5361,2022,Freddie Freeman,LAD,32,122,475,544,154,95,...,0.095,112.3,188,0.470,400,0.121,0.206,0.314,0.539,0.401
8,11493,2022,Manny Machado,SDP,29,114,437,490,131,77,...,0.096,112.4,167,0.487,343,0.129,0.240,0.265,0.447,0.340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,15117,2022,Hunter Dozier,KCR,30,101,373,407,88,54,...,0.076,110.7,103,0.372,277,0.138,0.276,0.239,0.389,0.304
119,2434,2022,Nelson Cruz,WSN,41,110,399,450,94,68,...,0.101,113.8,137,0.463,296,0.127,0.279,0.243,0.411,0.325
138,12179,2022,Maikel Franco,WSN,29,103,371,388,85,61,...,0.047,112.1,103,0.344,299,0.161,0.298,0.243,0.351,0.272
107,4314,2022,Joey Votto,CIN,38,91,322,376,66,36,...,0.111,111.9,91,0.404,225,0.141,0.266,0.216,0.389,0.322


In [52]:
# Add new columns for MVP and XBH+, then set the columns to match the trained set

current_year['MVP'] = 0
current_year['XBH+'] =  (((current_year['R'] + current_year['2B'] + current_year['3B'] + current_year['HR']) - (current_year['SO']))/current_year['PA'])

current_year = current_year[[ 'G',
 'AB',
 'PA',
 'H',
 '1B',
 '2B',
 '3B',
 'HR',
 'R',
 'RBI',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'SF',
 'SH',
 'SB',
 'AVG',
 'OBP',
 'SLG', 
 'OPS',
 'ISO',
 'BABIP',
 'wOBA',
 'wRAA',
 'wRC',
 'WAR',
 'Spd',
 'wRC+',
 'WPA',
 '-WPA',
 '+WPA',
 'RE24',
 'REW',
 'pLI',
 'WPA/LI',
 'Clutch',
 'BsR',
 'wSB',
 'Off',
 'AVG+',
 'OBP+',
 'SLG+',
 'ISO+',
 'BABIP+',
 'XBH+',
 'MVP']]

current_year

,G,AB,PA,H,1B,2B,3B,HR,R,RBI,...,BsR,wSB,Off,AVG+,OBP+,SLG+,ISO+,BABIP+,XBH+,MVP
1,120,448,529,133,65,20,0,48,100,105,...,1.3,1.9,57.9,123,128,170,247,109,0.062382,0
0,117,442,511,148,83,34,0,31,87,100,...,2.4,0.4,56.8,137,132,156,185,130,0.090020,0
3,115,438,489,132,72,33,1,26,61,82,...,-1.5,-1.1,32.1,123,116,140,166,102,0.122699,0
4,122,475,544,154,95,41,2,16,85,78,...,5.5,0.6,41.5,133,127,130,126,123,0.115809,0
8,114,437,490,131,77,31,1,22,77,76,...,3.0,0.5,31.9,123,118,132,146,116,0.071429,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,101,373,407,88,54,21,3,10,39,35,...,-7.6,-1.2,-10.7,98,96,100,103,101,-0.058968,0
119,110,399,450,94,68,16,0,10,46,60,...,-2.9,0.3,-9.1,96,100,88,74,101,-0.077778,0
138,103,371,388,85,61,15,0,9,31,39,...,-3.7,-0.1,-20.3,94,81,86,73,90,-0.051546,0
107,91,322,376,66,36,18,1,11,31,41,...,-4.1,-0.4,-8.0,84,101,92,106,88,-0.095745,0


In [60]:
NewX = current_year.drop(['MVP'],1).values


AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [58]:

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(NewX)

# Scale the data
X_train_scaled = X_scaler.transform(NewX)
X_test_scaled = X_scaler.transform(X_test)

pred_mvp = classifier.predict(NewX)

pred_mvp

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [59]:
NewX['MVP'] = pred_mvp
NewX

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices